In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image, ImageDraw, ImageFont
import json
import random
import gradio as gr
import queue

import clip
import os
from tqdm import tqdm
from utils.initialize_font_data import all_attributes, fox_text_four_lines, font_paths as roman_font_paths, retrieve_font_path
from utils.transform_image import draw_text_with_new_lines
from utils.cross_lingual_font_retrieval import inclusive_cjk_font_paths, roman_ref_font_paths, cjk_ref_font_paths

from cj_fonts import inclusive_fonts, fifty_fonts
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager

# If using GPU then use mixed precision training.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Must set jit=False for training
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

from torchvision.transforms.functional import pil_to_tensor, to_pil_image

exclusive_attributes:  ['capitals', 'cursive', 'display', 'italic', 'monospace', 'serif']


In [2]:
char_size = 150
font_dir = '../gwfonts'
cj_font_dir = '../all-fonts'
# cj_font_paths = [os.path.join(cj_font_dir, f) for f in os.listdir(cj_font_dir)]
cj_font_paths = inclusive_cjk_font_paths
fifty_cj_font_paths = [os.path.join(cj_font_dir, f) for f in fifty_fonts.split('\n') if f != '']

# add font
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

for font in font_manager.findSystemFonts(cj_font_dir):
    font_manager.fontManager.addfont(font)

ttf_list = font_manager.fontManager.ttflist

predicted_attributes = json.load(open('../attributeData/predicted_cj_font_attribute.json', 'r'))
attribute_to_indexes = {attribute: all_attributes.index(attribute) for attribute in all_attributes}

In [3]:
def choose_random_font_pairs(font_dir):
    font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
    font_path1 = random.choice(font_paths)
    font_path2 = random.choice(font_paths)
    while font_path1 == font_path2:
        font2_path2 = random.choice(font_paths)
    return font_path1, font_path2

In [4]:
def choose_font_pairs_in_order(font_dir):
    font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
    for font_path1 in font_paths:
        for font_path2 in font_paths:
            if font_path1 != font_path2:
                yield font_path1, font_path2

In [5]:
def choose_font_pairs_same_apper_times_with_queue(font_paths, appear_times=8, target_font_num=50, random_seed = 123):
    sorted_font_paths = sorted(font_paths)

    random.seed(random_seed)
    font_paths = sorted_font_paths[:target_font_num] * appear_times
    random.shuffle(font_paths)

    font_paths_queue = queue.Queue()
    for font_path in font_paths:
        font_paths_queue.put(font_path)
    flag = False
    count = 0
    # extract two font paths from font_paths_queue
    while True:
        if font_paths_queue.qsize() < 2:
            break
        if count > 1000:
            break

        font_path2 = font_paths_queue.get()
        if not flag:
            font_path1 = font_paths_queue.get()
        flag = False
        if font_path1 != font_path2:
            yield font_path1, font_path2
        else:
            count += 1
            font_paths_queue.put(font_path2)
            flag = True

In [6]:
def choose_reference_font(font_paths, appear_times=1, target_font_num=50, random_seed=123):
    sorted_font_paths = sorted(font_paths)
    random.seed(random_seed)
    font_paths = sorted_font_paths[:target_font_num] * appear_times
    random.shuffle(font_paths)

    font_paths_queue = queue.Queue()
    for font_path in font_paths:
        font_paths_queue.put(font_path)
    while True:
        if font_paths_queue.qsize() == 0:
            break

        font_path = font_paths_queue.get()
        yield font_path

In [7]:
cjk_text = '春夏\n秋冬'
roman_text = fox_text_four_lines

def draw_percentage_bar(current_num, total_num, size=(300, 10), background_color=(255, 255, 255), bar_color=(186, 251, 176), total_bar_color=(131, 237, 231)):
    if current_num == 0:
        return Image.new('RGB', size, background_color)

    percentage = min(current_num / total_num, 1)
    image = Image.new('RGB', size, background_color)
    draw = ImageDraw.Draw(image)
    bar_width = int(size[0] * percentage)
    bar_height = int(size[1] / 3)
    bar_height = int(0)
    draw.rectangle((0, bar_height, bar_width, int(size[1])), fill=bar_color)
    return image

In [8]:
char_size = 150
draw_percentage_bar(10, 10)
navigation_font_path = os.path.join(cj_font_dir, 'ipaexg.ttf')
navigation_font = ImageFont.truetype(navigation_font_path, char_size)
navigation_width = char_size * 15
navigation_height = char_size * 2

In [12]:
roman_char_size = 100
appearance_times = 4
target_font_num = 50
total_num = appearance_times * target_font_num // 2
print('total_num', total_num)
assert total_num == len(roman_ref_font_paths)
mode = 'CJK'
if mode == 'Roman':
  ref_font_generator = (font_path for font_path in roman_ref_font_paths)
  font_pair_generator = choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appearance_times, target_font_num=target_font_num, random_seed=123)
else:
  ref_font_generator = (font_path for font_path in cjk_ref_font_paths)
  font_pair_generator = choose_font_pairs_same_apper_times_with_queue(roman_font_paths, appear_times=appearance_times, target_font_num=target_font_num, random_seed=123)

total_num 100


In [13]:
current_num = 0
if mode == 'Roman':
  ref_text = roman_text
  text = cjk_text
  ref_char_size = roman_char_size
  option_char_size = char_size
else:
  ref_text = cjk_text
  text = roman_text
  ref_char_size = char_size
  option_char_size = roman_char_size

line_num = ref_text.count('\n') + 1
ref_default_width = int((ref_char_size+20) * len(ref_text) / line_num)
ref_default_height = int((ref_char_size+20) * line_num)
line_num = text.count('\n') + 1
default_width = int((option_char_size+20) * len(text) / line_num)
default_height = int((option_char_size+20) * line_num)


default_ref_font_path = next(ref_font_generator)
default_ref_font = ImageFont.truetype(default_ref_font_path, ref_char_size)
default_font_path1, default_font_path2 = next(font_pair_generator)
default_font1 = ImageFont.truetype(default_font_path1, option_char_size)
default_font2 = ImageFont.truetype(default_font_path2, option_char_size)


global_font_name1 = os.path.splitext(os.path.basename(default_font_path1))[0]
global_font_name2 = os.path.splitext(os.path.basename(default_font_path2))[0]
global_ref_font_name = os.path.splitext(os.path.basename(default_ref_font_path))[0]
print(f'{global_font_name1}, {global_font_name2}, {global_ref_font_name}')

Aldrich, Bello-Pro, 001Shirokuma-Regular


In [14]:
annotation_result = []
def builder(result, name, ref_text=ref_text, text=text, char_size=char_size, total_num=total_num, mode=mode):
        
    global global_ref_font_name, global_font_name1, global_font_name2, current_num, target_attribute_count, annotation_result, ref_font_generator, font_pair_generator


    # set annotation_result
    annotation_result.append((global_ref_font_name, global_font_name1, global_font_name2, result))

    #font_path1, font_path2 = choose_random_font_pairs(font_dir)
    current_num += 1
    percentage_image = draw_percentage_bar(current_num, total_num=total_num)
    try:
        ref_font_path = next(ref_font_generator)
        font_path1, font_path2 = next(font_pair_generator)
        # set up percentage bar
    except StopIteration:
        print('StopIteration')

        rator_name = name.replace(' ', '_')
        output_dir = '../attributeData/cross-lingual-outputs'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        output_path = os.path.join(output_dir, f'{rator_name}-{mode}-{target_font_num}.json')
        if not os.path.exists(output_path):
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(annotation_result, f, ensure_ascii=False)
        image1 = draw_text_with_new_lines('finish', navigation_font, ref_default_width, ref_default_height)
        image2 = draw_text_with_new_lines('thank', navigation_font, ref_default_width, ref_default_height)
        image3 = draw_text_with_new_lines('you', navigation_font, ref_default_width, ref_default_height)
        percentage_image = draw_percentage_bar(current_num, total_num=total_num)
        return image1, image2, image3, percentage_image

    print(current_num, ref_font_path, font_path1, font_path2)
    ref_font = ImageFont.truetype(ref_font_path, ref_char_size)
    font1 = ImageFont.truetype(font_path1, option_char_size)
    font2 = ImageFont.truetype(font_path2, option_char_size)
    ref_font_name = os.path.splitext(os.path.basename(ref_font_path))[0]
    font_name1 = os.path.splitext(os.path.basename(font_path1))[0]
    font_name2 = os.path.splitext(os.path.basename(font_path2))[0]
    global_font_name1 = font_name1
    global_font_name2 = font_name2
    global_ref_font_name = ref_font_name

    image1 = draw_text_with_new_lines(ref_text, ref_font, ref_default_width, ref_default_height)
    image2 = draw_text_with_new_lines(text, font1, default_width, default_height)
    image3 = draw_text_with_new_lines(text, font2, default_width, default_height)
    return image1, image2, image3, percentage_image

with gr.Blocks() as demo:

    with gr.Row():
        name = gr.Text(value='Your Name', label='Name', interactive=True)
    with gr.Row():
        percentage_image = gr.Image(value=draw_percentage_bar(0, total_num=total_num), label='Progress', interactive=False, show_label=False)
    with gr.Row():
        with gr.Column(scale=0.4):
            pass
        with gr.Column(scale=0.2):
            image1 = gr.Image(value=draw_text_with_new_lines(ref_text, default_ref_font, ref_default_width, ref_default_height), label='Reference', interactive=False)
        with gr.Column(scale=0.4):
            pass
    with gr.Row():
        with gr.Column(scale=0.3):
            pass
        with gr.Column(scale=0.2):
            image2 = gr.Image(value=draw_text_with_new_lines(text, default_font1, default_width, default_height), label='Font A', interactive=False)
        with gr.Column(scale=0.2):
            image3 = gr.Image(value=draw_text_with_new_lines(text, default_font2, default_width, default_height), label='Font B', interactive=False)
        with gr.Column(scale=0.3):
            pass
    with gr.Row():
        with gr.Column(scale=0.4):
            pass
        with gr.Column(scale=0.1):
            check = gr.Radio(choices=['Font A', 'Font B'], value = 'Font A', label=f'', interactive=True)
        with gr.Column(scale=0.2):
            next_button  = gr.Button(value='Next', interactive=True)
    next_button.click(builder, inputs=[check, name], outputs=[image1, image2, image3, percentage_image], show_progress=True)



demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860


Running on public URL: https://f55b648d18bdc17b48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1 ../all-fonts/AsebiMin-Light.otf /home/yuki/learn-computer-vision/gwfonts/Amethysta-Regular.ttf /home/yuki/learn-computer-vision/gwfonts/Arizonia-Regular.ttf
2 ../all-fonts/Boku2-Regular.otf /home/yuki/learn-computer-vision/gwfonts/Andada-Bold.ttf /home/yuki/learn-computer-vision/gwfonts/ARSMaquetteWebOne.ttf
3 ../all-fonts/chinese-handwriting-style.ttf /home/yuki/learn-computer-vision/gwfonts/CabinCondensed-Medium.ttf /home/yuki/learn-computer-vision/gwfonts/BilboSwashCaps-Regular.ttf
4 ../all-fonts/站酷庆科黄油体.ttf /home/yuki/learn-computer-vision/gwfonts/ARSMaquetteWebOne.ttf /home/yuki/learn-computer-vision/gwfonts/AnonymousPro.ttf
5 ../all-fonts/dfpop2w9.ttc /home/yuki/learn-computer-vision/gwfonts/Bevan.ttf /home/yuki/learn-computer-vision/gwfonts/BadScript-Regular.ttf
6 ../all-fonts/HarmonyOS_Sans_SC_Light.ttf /home/yuki/learn-computer-vision/gwfonts/CabinCondensed-Medium.ttf /home/yuki/learn-computer-vision/gwfonts/Bevan.ttf
7 ../all-fonts/jianshuihei.TTF /home/yuki/learn-computer-

In [ ]:
text = '春夏\n秋冬'
top5_attributes = ['thin', 'calm', 'sloppy', 'complex', 'strong',]
bottom5_attributes = ['happy', 'fresh', 'gentle', 'sharp', 'technical',]
#target_attributes = top5_attributes
target_attributes = ['thin', 'calm', 'sloppy', 'sharp', 'technical', 'traditional', 'Japanese_style', 'robust']
target_attribute = target_attributes[0]
fifty_cj_font_paths = [os.path.join(cj_font_dir, f) for f in fifty_fonts.split('\n') if f != '']
target_font_num = 50
#target_font_num = 10
appear_times = 2
share = True


line_num = text.count('\n') + 1
default_width = int((char_size+50) * len(text) / line_num)
default_height = int((char_size+50) * line_num)
#font_paris_generator = choose_font_pairs_in_order(cj_font_dir)

# generate random seed by converting target_attribute to int
random_seed = int.from_bytes(target_attribute.encode(), 'little')

font_paris_generator = choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appear_times, target_font_num=target_font_num, random_seed = random_seed)
total_pair_num = sum([1 for _ in font_paris_generator])
print('total_pair_num', total_pair_num)
font_paris_generator = choose_font_pairs_same_apper_times_with_queue(fifty_cj_font_paths, appear_times=appear_times, target_font_num=target_font_num, random_seed = random_seed)

default_font_path1, default_font_path2 = next(font_paris_generator)
print(default_font_path1)
default_font1 = ImageFont.truetype(default_font_path1, char_size)
default_font2 = ImageFont.truetype(default_font_path2, char_size)
navigation_font_path = os.path.join(cj_font_dir, 'ipaexg.ttf')
navigation_font = ImageFont.truetype(navigation_font_path, char_size)
navigation_width = char_size * 15
navigation_height = char_size * 2

global_font_name1 = os.path.splitext(os.path.basename(default_font_path1))[0]
global_font_name2 = os.path.splitext(os.path.basename(default_font_path2))[0]

annotation_result = []
target_attribute_count = 0
current_num = 0
save_flags = []

def create_attribute_markdown(target_attribute):
    #return f'<div style="font-size: 50px; text-align: center">Which font is more {target_attribute}?</div>'
#     return f'Which is more {target_attribute}?'
    return f'{target_attribute}?'




../all-fonts/JP_Ronde-B_square.otf ../all-fonts/SanariFontB001.ttf
../all-fonts/ZenOldMincho-Black.ttf ../all-fonts/HanyiSentyPastel.ttf
../all-fonts/crayon_1-1.ttf ../all-fonts/SentyCaramel.ttf
../all-fonts/kiloji_d.ttf ../all-fonts/irohamaru-Medium.ttf
../all-fonts/Tanugo-TTF-Regular.ttf ../all-fonts/YuseiMagic-Regular.ttf
../all-fonts/chogokubosogothic_5.ttf ../all-fonts/g_pencilkaisho_free.ttf
../all-fonts/HanyiSentyPastel.ttf ../all-fonts/ゆず ポップ A [M] Light.ttf
../all-fonts/HachiMaruPop-Regular.ttf ../all-fonts/crayon_1-1.ttf
../all-fonts/KFhimaji.otf ../all-fonts/Corporate-Logo-Medium-ver3.otf
../all-fonts/kiloji_d.ttf ../all-fonts/Kazesawa-ExtraLight.ttf
../all-fonts/Corporate-Logo-Medium-ver3.otf ../all-fonts/chinese-handwriting-style.ttf
../all-fonts/SentyGoldSand.ttf ../all-fonts/Chalk-S-JP.otf
../all-fonts/g_comickoin_freeL.ttf ../all-fonts/35.ttf
../all-fonts/g_comickoin_freeB.ttf ../all-fonts/ZCOOLXiaoWei-Regular.ttf
../all-fonts/KodomoRounded.otf ../all-fonts/M+A1_heav

In [ ]:
name='Takahori_Hosokawa'
target_attribute = 'thin'
with open(f'../attributeData/outputs/{name}-{target_attribute}-{target_font_num}.json', 'r') as f:
    results = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../attributeData/outputs/Takahori_Hosokawa-thin-50.json'

In [ ]:
len(results)

50